In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
parentdir = os.path.dirname(os.getcwd())
sys.path.append(parentdir)
sys.path.append(os.path.join(parentdir, 'py'))
import ncreate3d as nc
from config import cfg

### Create nozzle geometry. This will export mesh variables

In [ ]:
help(nc.NozVars.__init__)

In [ ]:
help(nc.genericMesh)

In [ ]:
fg = nc.genericMesh(os.path.join(cfg.path.c, 'LapRDTall'), npts=50, bathHeight=10, bathWidth=16) # generate mesh files
fg.plot.savefig(os.path.join(cfg.path.fig, 'yieldingsweep','plots', 'LapRDTall', 'nozzleDiagramTall.png')) # export the diagram of the nozzle

In [ ]:
fg = nc.genericMesh(os.path.join(cfg.path.c, 'conical'), npts=50, bathDepth=16, bathHeight=10, nozzleAngle=10) # generate mesh files
fg.plot.savefig(os.path.join(cfg.path.fig, 'conicalNozzle', 'plots', 'nozzleDiagram.svg')) # export the diagram of the nozzle

In [ ]:
fg = nc.genericMesh(os.path.join(cfg.path.c, 'nonozzle')) # generate mesh files

## Generate simulation files

Usage:
<br>e.g. To create a Newtonian fluid of viscosity 1000 Pa s and density 1100 kg/m^3:
<br>&nbsp;&nbsp;&nbsp;&nbsp;`myFluid = nc.Fluid(nu=1000, rho=1100, units='Pas')`
<br>or to create a Newtonian fluid of viscosity 1000 Pa s and density 1000 kg/m^3:
<br>&nbsp;&nbsp;&nbsp;&nbsp;`myFluid = nc.Fluid(nu=1000, units='Pa')`
<br>e.g. To create a Herschel-Bulkley fluid of plateau viscosity 4 Pas, yield stress 1 Pa, consistency index 10, power law index 0.5:
<br>&nbsp;&nbsp;&nbsp;&nbsp;`myFluid = nc.Fluid(nu0=4, tau0=1, k=10, n=0.5, units="Pa")`
<br>You can also give the fluid a label, but the label won't be used during file generation. You need to use it separately.
<br>&nbsp;&nbsp;&nbsp;&nbsp;`myFluid = nc.Fluid(label='myLabel', nu0=4, tau0=1, k=10, n=0.5, units="Pa")`
<br><br>
<br>Sigma must be in J/m^2 (e.g. 0.04 J/m^2)
<br>To create a simulation folder, use genericExport.
<br>&nbsp;&nbsp;&nbsp;&nbsp;`nc.genericExport(folderNumber, fluid1, fluid2, surfaceTension, parentFolder, exportMesh=False)`
<br>To create a simulation folder with its own mesh files, use exportMesh=True.
<br>&nbsp;&nbsp;&nbsp;&nbsp;`nc.genericExport(folderNumber, fluid1, fluid2, surfaceTension, parentFolder, exportMesh=True)`

## conicals

### determine bath size

In [ ]:
nv0 = nc.NozVars(nozzleAngle=0, bathHeight=10, bathWidth=16)
nv30 = nc.NozVars(nozzleAngle=30, bathHeight=10, bathWidth=16)
for n in [nv0, nv30]:
    print([getattr(n, s) for s in ['bw', 'bh', 'bd']])

In [ ]:
nv0 = nc.NozVars(nozzleAngle=0, bathHeight=7, bathWidth=16)
nv30 = nc.NozVars(nozzleAngle=30, bathHeight=7, bathWidth=16)
for n in [nv0, nv30]:
    print([getattr(n, s) for s in ['bw', 'bh', 'bd']])

### generate starting files

#### speed/visc sweep

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 144
sigma = 0
fHB = nc.Fluid(units="Pa", label='', tau0=10, k=3.75, n=0.45, nu0=10**4)
fnewt = nc.Fluid(units="Pa", label='', nu=10)
for speed in [2.5, 5, 7.5, 10]:
    for f in [fHB]:
        for angle in [0, 5, 10, 15, 20, 25, 30]:
            nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                             , bathHeight=10, bathWidth=22,  nozzleAngle=angle, slurmFolder='/working/lmf1/conical'
                            , vink=speed, vbath=speed)
            ii+=1
        for angle in [0, 5, 10, 15, 20, 25, 30]:
            ii+=1

#### Newtonian angle sweep, 10 Pa.s

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 200
sigma = 0
fnewt = nc.Fluid(units="Pa", label='', nu=10)
for f in [fnewt]:
    for angle in [0, 5, 10, 15, 20, 25, 30]:
        nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=angle, slurmFolder='/working/lmf1/conical'
                        , vink=10, vbath=10)
        ii+=1

#### HB diameter sweep, absolute mesh 

In [ ]:
gdot = 10/0.903
[10/gdot+k*gdot**(0.45-1) for k in [0.375, 3.75, 37.5, 375]]

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 207
sigma = 0
fHB = nc.Fluid(units="Pa", label='', tau0=10, k=3.75, n=0.45, nu0=10**4)
innerdiams = [0.838,  0.413, 0.26]
widths = [0.216,  0.152, 0.127]
for f in [fHB]:
    for angle in [0, 15]:
        for i,diam in enumerate(innerdiams):
            nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=angle, slurmFolder='/working/lmf1/conical'
                        , vink=10, vbath=10, nozzleInnerWidth=diam, nozzleThickness=widths[i])
            ii+=1

#### HB k sweep

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 213
sigma = 0
fHBlist = [nc.Fluid(units="Pa", label='', tau0=10, k=ki, n=0.45, nu0=10**4) for ki in [0.375, 37.5, 375]]
for f in fHBlist:
    for angle in [0, 15]:
        nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=angle, slurmFolder='/working/lmf1/conical'
                        , vink=10, vbath=10)
        ii+=1

#### Newtonian angle sweep, 1 Pa.s

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 219
sigma = 0
fnewt = nc.Fluid(units="Pa", label='', nu=1)
for f in [fnewt]:
    for angle in [0, 5, 10, 15, 20, 25, 30]:
        nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=angle, slurmFolder='/working/lmf1/conical'
                        , vink=10, vbath=10)
        ii+=1

#### HB diameter sweep, relative mesh

In [ ]:
# diameter, mesh size relative to nozzle diameter
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 226
sigma = 0
fHB = nc.Fluid(units="Pa", label='', tau0=10, k=3.75, n=0.45, nu0=10**4)
innerdiams = [0.838,  0.413, 0.26]
widths = [0.216,  0.152, 0.127]
for f in [fHB]:
    for angle in [0, 15]:
        for i,diam in enumerate(innerdiams):
            nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=angle, slurmFolder='/working/lmf1/conical'
                        , vink=10, vbath=10, nozzleInnerWidth=diam, nozzleThickness=widths[i])
            ii+=1

#### HB speed sweep, 15 degree nozzle

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 232
sigma = 0
f = nc.Fluid(units="Pa", label='', tau0=10, k=3.75, n=0.45, nu0=10**4)
for speed in [5, 15]:
    for angle in [15]:
        nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=angle, slurmFolder='/working/lmf1/conical'
                        , vink=speed, vbath=speed)
        ii+=1

#### Newtonian diameter sweep, mixed mesh

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 234
sigma = 0
f = nc.Fluid(units="Pa", label='', nu=10)
innerdiams = [ 0.413, 0.26]
widths = [ 0.152, 0.127]
nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=0, slurmFolder='/working/lmf1/conical'
                        , vink=10, vbath=10, nozzleInnerWidth=0.838, nozzleThickness=0.216)
ii+=1
for i,diam in enumerate(innerdiams):
    nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=0, slurmFolder='/working/lmf1/conical'
                        , vink=10, vbath=10, meshSize=0.2, nozzleInnerWidth=diam, nozzleThickness=widths[i])
    ii+=1

#### Newtonian viscosity sweep, 15 degree nozzle

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 237
sigma = 0

for visc in [0.1, 100]:
    fnewt = nc.Fluid(units="Pa", label='', nu=visc)
    for angle in [15]:
        nc.genericExport(ii, fnewt, fnewt, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=angle
                         , slurmFolder='/working/lmf1/conical'
                        , vink=10, vbath=10)
        ii+=1

#### Newtonian diameter sweep, relative mesh

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 239
sigma = 0
f = nc.Fluid(units="Pa", label='', nu=10)
innerdiams = [ 0.413, 0.26]
widths = [ 0.152, 0.127]
for i,diam in enumerate(innerdiams):
    nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=0, slurmFolder='/working/lmf1/conical'
                        , vink=10, vbath=10, nozzleInnerWidth=diam, nozzleThickness=widths[i])
    ii+=1

#### Newtonian diameter sweep, relative speed, mixed mesh

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 241
sigma = 0
f = nc.Fluid(units="Pa", label='', nu=10)
innerdiams = [ 0.413, 0.26]
widths = [ 0.152, 0.127]
nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=0, slurmFolder='/working/lmf1/conical'
                        , vink=10*838/0.603, vbath=10*838/0.603, nozzleInnerWidth=0.838, nozzleThickness=0.216)
ii+=1
for i,diam in enumerate(innerdiams):
    nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=0, slurmFolder='/working/lmf1/conical'
                        , vink=10*diam/0.603, vbath=10*diam/0.603, meshSize=0.2, nozzleInnerWidth=diam, nozzleThickness=widths[i])
    ii+=1

#### Newtonian diameter sweep, relative mesh, 2000000 maxglobalcells

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 244
sigma = 0
f = nc.Fluid(units="Pa", label='', nu=10)
innerdiams = [0.838,  0.413, 0.26]
widths = [0.216,  0.152, 0.127]
for i,diam in enumerate(innerdiams):
    nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=0, slurmFolder='/working/lmf1/conical'
                        , vink=10, vbath=10, nozzleInnerWidth=diam, nozzleThickness=widths[i])
    ii+=1

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 247
sigma = 0
f = nc.Fluid(units="Pa", label='', nu=10)
innerdiams = [ 0.413, 0.26]
widths = [0.152, 0.127]
for i,diam in enumerate(innerdiams):
    nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=0, slurmFolder='/working/lmf1/conical'
                        , vink=10, vbath=10, nozzleInnerWidth=diam, nozzleThickness=widths[i]
                    , meshSize=0.2)
    ii+=1

#### HB diameter sweep, relative speed, mixed mesh

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 252
sigma = 0
f = nc.Fluid(units="Pa", label='', tau0=10, k=3.75, n=0.45, nu0=10**4)
innerdiams = [ 0.413, 0.26]
widths = [ 0.152, 0.127]
nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=0, slurmFolder='/working/lmf1/conical'
                        , vink=10*0.838/0.603, vbath=10*0.838/0.603, nozzleInnerWidth=0.838, nozzleThickness=0.216)
ii+=1
for i,diam in enumerate(innerdiams):
    nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=0, slurmFolder='/working/lmf1/conical'
                        , vink=10*diam/0.603, vbath=10*diam/0.603, meshSize=0.2, nozzleInnerWidth=diam, nozzleThickness=widths[i])
    ii+=1

#### Newtonian diameter sweep, relative speed, mixed mesh

In [ ]:
topfolder = os.path.join(cfg.path.c, 'conical') # this is the parent folder that holds everything
ii = 255
sigma = 0
f = nc.Fluid(units="Pa", label='', nu=10)
nc.genericExport(ii, f, f, sigma, topfolder, exportMesh=True, folderBase="cn"
                         , bathHeight=10, bathWidth=16,  nozzleAngle=0, slurmFolder='/working/lmf1/conical'
                        , vink=10*0.838/0.603, vbath=10*0.838/0.603, nozzleInnerWidth=0.838, nozzleThickness=0.216)

## Laponite

In [ ]:
topfolder = os.path.join(cfg.path.c, 'LapRDTall') # this is the parent folder that holds everything
ii = 1136
sigma = 0

LapRD225 = nc.Fluid(units="Pa", label='LapRD_2.25', tau0=0.21394, k=3.248436, n=0.14201, nu0=4.05384)
LapRD250 = nc.Fluid(units="Pa", label='LapRD_2.50', tau0=0.82365, k=9.692119, n=0.112515, nu0=11.8239)
LapRD275 = nc.Fluid(units="Pa", label='LapRD_2.75', tau0=2.07291, k=22.74276, n=0.054603, nu0=22.6813)
LapRD300 = nc.Fluid(units="Pa", label='LapRD_3.00', tau0=6.72765, k=75.69318, n=0.022874, nu0=73.66575)
LapRD350 = nc.Fluid(units="Pa", label='LapRD_3.50', tau0=26.8672, k=182.7635, n=0, nu0=99.1863)
LapRD400 = nc.Fluid(units="Pa", label='LapRD_4.00', tau0=93.9063, k=181.8893, n=0.021436, nu0=219.404)

inks = [LapRD225, LapRD250, LapRD275, LapRD300, LapRD350, LapRD400]

for link in inks:
    for lsup in inks:
        nc.genericExport(ii, lsup, link, sigma, topfolder, exportMesh=False)
        ii+=1

## Viscosity sweep

#### fine Newtonian resolution sweep

In [ ]:
topfolder = os.path.join(cfg.path.c, 'NewtFine') # this is the parent folder that holds everything
if not os.path.exists(topfolder):
    os.mkdir(topfolder)
ii = 700

newtinks = [nc.Fluid(units="Pa", label='10^'+str(i), nu=10**i) for i in np.arange(-0.5, 5.5,1)]
newtsups = [nc.Fluid(units="Pa", label='10^'+str(i), nu=10**i) for i in [1,2]]

for sigma in [0, 0.04]:
    for link in newtinks:
        for lsup in newtsups:
            nc.genericExport(ii, lsup, link, sigma, topfolder, exportMesh=False, slurmFolder='/working/lmf1/NewtFine')
            ii+=1

#### find HB resolution sweep

In [ ]:
topfolder = os.path.join(cfg.path.c, 'HBHBFine') # this is the parent folder that holds everything
if not os.path.exists(topfolder):
    os.mkdir(topfolder)
ii = 724

HBinks = [nc.Fluid(units="Pa", label='10^'+str(i), nu0=10**i, tau0=10, k=3.75, n=0.45) for i in np.arange(-0.5, 5.5,1)]
HBsups = [nc.Fluid(units="Pa", label='10^'+str(i), nu0=10**i, tau0=10, k=3.75, n=0.45) for i in [1,2]]

for sigma in [0, 0.04]:
    for link in HBinks:
        for lsup in HBsups:
            nc.genericExport(ii, lsup, link, sigma, topfolder, exportMesh=False, slurmFolder='/working/lmf1/HBHBFine')
            ii+=1